In [1]:
# https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/
# Continued from 2.
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [2]:
base_model = VGG16(weights='imagenet', include_top=False)
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [4]:
# loading the trained weights
model.load_weights("weight.hdf5")
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [12]:
f = open("test/data/ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [6]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [27]:
def clear(path):
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)

def predict(indir, test, tmp="temp"):
    
    # creating two lists to store predicted and actual tags
    predict = []
    actual = []

    os.makedirs(tmp, exist_ok=True)
    
    # for loop to extract frames from each test video
    for i in tqdm(range(test.shape[0])):
        count = 0
        
        videoFile = test['video_name'][i]
        inpath = os.path.join(indir, videoFile)
        if not os.path.isfile(inpath):
            raise Exception("Path does not exist {}".format(inpath))
            
        clear(tmp)
        cap = cv2.VideoCapture(inpath)   # capturing the video from the given path
        try:
            frameRate = cap.get(5) #frame rate
            x=1
  
            while(cap.isOpened()):
                frameId = cap.get(1) #current frame number
                ret, frame = cap.read()
                if (ret != True):
                    break
                if (frameId % math.floor(frameRate) == 0):
                    # storing the frames of this particular video in temp folder
                    filename ='temp/' + "_frame%d.jpg" % count;count+=1
                    cv2.imwrite(filename, frame)
        finally:
            cap.release()

        # reading all the frames from temp folder
        images = glob("temp/*.jpg")

        prediction_images = []
        for i in range(len(images)):
            img = image.load_img(images[i], target_size=(224,224,3))
            img = image.img_to_array(img)
            img = img/255
            prediction_images.append(img)

        # converting all the frames for a test video into numpy array
        prediction_images = np.array(prediction_images)
        # extracting features using pre-trained model
        prediction_images = base_model.predict(prediction_images)
        
        # converting features in one dimensional array
        prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
        # predicting tags for each array
        prediction = model.predict_classes(prediction_images)
        
        # appending the mode of predictions in predict list to assign the tag to the video
        predict.append(y.columns.values[s.mode(prediction)[0][0]])
        # appending the actual tag of the video
        actual.append(videoFile.split('/')[1].split('_')[1])
        
    return predict, actual

In [28]:
pred, actual = predict("test/data/UCF-101", test)

100%|██████████| 3783/3783 [52:19<00:00,  1.21it/s]  


In [31]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(pred, actual)*100

87.20592122653979